In [1]:
import torch
import copy
import numpy as np
from models.DTWRNN import Model as DTWRNN_Model
from utils.evaluators import compute_acc, compute_ave_batch_loss
from datasets.process import ecg5000_reading
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

shaplets = torch.load('datasets/ECG5000/shaplets/shaplets_{10.10.10.10.10}_{random_select}.pt')


X, Y = ecg5000_reading()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=RANDOM_SEED, stratify=Y)
X_verify, X_test, Y_verify, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=RANDOM_SEED, stratify=Y_test)

class Kaggle_ECG(Dataset):
    def __init__(self, X, Y):
        self.x = X
        self.y = Y
        self.length = len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.length
    
ds_train = Kaggle_ECG(X_train, Y_train)
ds_verify = Kaggle_ECG(X_verify, Y_verify)
ds_test = Kaggle_ECG(X_test, Y_test)

/root/autodl-tmp/datasets/process.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train.append(test, ignore_index=True)


In [2]:
_n_epoch = 80
_batch_size = 200
train_data_loader = DataLoader(dataset=ds_train, batch_size=_batch_size, shuffle=True)
verify_data_loader = DataLoader(dataset=ds_verify, batch_size=_batch_size, shuffle=True)
test_data_loader = DataLoader(dataset=ds_test, batch_size=_batch_size, shuffle=True)

model = DTWRNN_Model(shaplets=shaplets.reshape(50, 50, 1), batch_size=_batch_size)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [3]:
for epoch in range(_n_epoch):
    
    if epoch == 0:
        acc, prec, f1 = compute_acc(model, test_data_loader, labels=[0, 1, 2, 3, 4])
        print('🌟🌟🌟 Zero-shot Result: \tAccuracy: {:.6f} \tPrecision: {:.6f} \tF1-score: {:.6f}'.format(
            acc, prec, f1
        ))
    
    train_loss = 0
    for i, data in enumerate(train_data_loader):
        optimizer.zero_grad()
        data_t, target = data[0].cuda(), data[1].reshape(_batch_size)
        o_ = model(data_t)
        loss = criterion(torch.log(o_), target.cuda())
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * _batch_size
        
    epoch_train_loss = train_loss / len(train_data_loader)
    
    epoch_verify_loss = compute_ave_batch_loss(model, verify_data_loader, criterion)
    
    epoch_test_loss = compute_ave_batch_loss(model, test_data_loader, criterion)
    
    acc, prec, f1 = compute_acc(model, test_data_loader, labels=[0, 1, 2, 3, 4])
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}  \tTesting Loss: {:.6f} '
         ' \n\t\tAccuracy: {:.6f} \tPrecision: {:.6f} \tF1-score: {:.6f}'.format(
        epoch, 
        epoch_train_loss,
        epoch_verify_loss,
        epoch_test_loss,
        acc, prec, f1
    ))

🌟🌟🌟 Zero-shot Result: 	Accuracy: 0.769000 	Precision: 0.472112 	F1-score: 0.478949
Epoch: 0 	Training Loss: 152.162183 	Validation Loss: 118.338417  	Testing Loss: 116.794966  
		Accuracy: 0.877000 	Precision: 0.548038 	F1-score: 0.559454
Epoch: 1 	Training Loss: 107.271086 	Validation Loss: 96.367968  	Testing Loss: 91.829954  
		Accuracy: 0.904000 	Precision: 0.594831 	F1-score: 0.595537
Epoch: 2 	Training Loss: 89.826548 	Validation Loss: 84.621608  	Testing Loss: 79.287964  
		Accuracy: 0.921000 	Precision: 0.640338 	F1-score: 0.639345
Epoch: 3 	Training Loss: 80.096590 	Validation Loss: 77.704819  	Testing Loss: 70.996743  
		Accuracy: 0.924000 	Precision: 0.674153 	F1-score: 0.658400
Epoch: 4 	Training Loss: 73.077493 	Validation Loss: 73.062605  	Testing Loss: 66.439807  
		Accuracy: 0.929000 	Precision: 0.709430 	F1-score: 0.674438
Epoch: 5 	Training Loss: 68.072580 	Validation Loss: 68.893248  	Testing Loss: 62.355544  
		Accuracy: 0.932000 	Precision: 0.719276 	F1-score: 0.67